In [4]:
import sys
import os
import pandas as pd
import numpy as np
import configparser
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt
import lightgbm as lgb
import lightgbm as lgb
from plotnine import ggplot, aes, geom_boxplot, labs, scale_color_manual, geom_point
import configparser

project_root = "/home/dan1dr/zrive-ds-4q24-churn"

# Define the project root path
current_wd = os.getcwd()

# Change the working directory if necessary
if current_wd != project_root:
    print(f"Changing working directory from {current_wd} to {project_root}")
    os.chdir(project_root)
else:
    print("Already in the correct path")

# Add 'src' directory to sys.path
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    print(f"Adding {src_path} to sys.path")
    sys.path.insert(0, src_path)

# Import the modules
from db_connectors.bigquery_service import BigqueryService
from data_gathering import data_gathering
from utils.logger import get_logger
from data_cleaning import data_cleaning
from column_config import users_cols, diff_cols, keep_cols, transform_cols, target_col


config = configparser.ConfigParser()
config.read("src/params.ini")

logger = get_logger(__name__)


def get_initial_params():
    """
    Loads all relevant parameters into a global variable. These parameters are then accessible to other functions in the script.
    Each function can consume the parameters it requires for its operation.
    This approach ensures centralized management and consistency of parameters across different functions.

    Returns:
        None: This function does not return a value but populates a global variable
        with necessary parameters.
    """
    global train_from, train_to, end_date, logistic_regression_params, lightgbm_params

    config = configparser.ConfigParser()
    config.read("src/params.ini")

    train_from = config.get("PARAMS", "train_from")
    train_to = config.get("PARAMS", "train_to")
    end_date = config.get("PARAMS", "end_date")
    # Load Logistic Regression parameters
    logistic_regression_params = {
        "penalty": config.get("LOGISTIC_REGRESSION", "penalty", fallback="l2"),
        "C": config.getfloat("LOGISTIC_REGRESSION", "C", fallback=1.0),
        "solver": config.get("LOGISTIC_REGRESSION", "solver", fallback="saga"),
        "max_iter": config.getint("LOGISTIC_REGRESSION", "max_iter", fallback=10000),
    }

    # Load LightGBM parameters
    lightgbm_params = {
        "boosting_type": config.get("LIGHTGBM", "boosting_type", fallback="gbdt"),
        "num_leaves": config.getint("LIGHTGBM", "num_leaves", fallback=12),
        "max_depth": config.getint("LIGHTGBM", "max_depth", fallback=-1),
        "learning_rate": config.getfloat("LIGHTGBM", "learning_rate", fallback=0.005),
        "n_estimators": config.getint("LIGHTGBM", "n_estimators", fallback=100),
        "random_state": config.getint("LIGHTGBM", "random_state", fallback=500),
        "colsample_bytree": config.getfloat(
            "LIGHTGBM", "colsample_bytree", fallback=0.64
        ),
        "subsample": config.getfloat("LIGHTGBM", "subsample", fallback=0.7),
        "reg_alpha": config.getint("LIGHTGBM", "reg_alpha", fallback=0),
        "reg_lambda": config.getint("LIGHTGBM", "reg_lambda", fallback=1),
    }


get_initial_params()
save_curves_path = "src/models"
save_features_path = "src/features"
save_target_path = "src/target"

Already in the correct path


In [5]:
import pandas as pd
import numpy as np
from column_config import users_cols, diff_cols, keep_cols, transform_cols, target_col


def feature_computation(
    clean_data: pd.DataFrame,
    train_from: str,
    train_to: str,
    logger,
    keep_gap_month_churns: bool = False,
    save_features_path: str = "",
    save_target_path: str = "",
) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):
    """
    Split data into train and test features set, aggregate the data into historical behavior for those cols needed.
    It also joins it with already calculated features, and extract the needed target from 2 months ahead.
    Args:
        clean_data: The cleaned dataset with customer, month, and payment information.
        train_from: The starting date of the training period.
        train_to: The ending date of the training period.

    Returns:
        DataFrame: Pandas DataFrame with computed features for model training.
        Series: Pandas Series representing the target variable for train set.
        DataFrame: Pandas DataFrame with computed features for model testing.
        Series: Pandas Series representing the target variable for test set.
    """
    logger.info("Starting feature computation")

    # TO-DO: Catch exceptions
    # TO-DO: Potential unit tests validating same length for features/targets
    # TO-DO: Instead of defining the cols every time import them somewhere else (they're need in data_cleaning also)

    # Convert the train_from and train_to to datetime
    train_from_dt = pd.to_datetime(train_from)
    train_to_dt = pd.to_datetime(train_to)

    # Filter train and test data before feature computation
    test_from_dt = train_from_dt + pd.DateOffset(months=1)
    test_to_dt = train_to_dt + pd.DateOffset(months=1)
    target_train_month = train_to_dt + pd.DateOffset(months=2)
    target_test_month = test_to_dt + pd.DateOffset(months=2)

    logger.info(
        f"Train computation from {train_from_dt} to {train_to_dt}. Target for {target_train_month}"
    )
    logger.info(
        f"Test computation from {test_from_dt} to {test_to_dt}. Target for {target_test_month}"
    )

    compute_ready_data = clean_data[
        users_cols + transform_cols + keep_cols + diff_cols + target_col
    ].copy()
    train_df, test_df = split_train_test(
        compute_ready_data, train_from_dt, train_to_dt, test_from_dt, test_to_dt
    )

    # We would train with previous users for train, but we need to remove them from test
    # as they wouldn't porbably
    previous_churned_users_train = train_df[
        (train_df["date"] <= train_to_dt) & (train_df[target_col[0]] > 0)
    ]["customer_id"].unique()
    previous_churned_users_test = test_df[
        (test_df["date"] <= test_to_dt) & (test_df[target_col[0]] > 0)
    ]["customer_id"].unique()
    train_df = train_df[~train_df["customer_id"].isin(previous_churned_users_train)]
    test_df = test_df[~test_df["customer_id"].isin(previous_churned_users_test)]

    logger.info(
        f"Removing {len(previous_churned_users_train)} previous churned users from train set"
    )
    logger.info(
        f"Removing {len(previous_churned_users_test)} previous churned users from test set"
    )
    logger.info(f"Unique customers in train: {train_df['customer_id'].nunique()}")
    logger.info(f"Unique customers in test: {test_df['customer_id'].nunique()}")

    logger.info("Starting features and target computation")
    logger.info(f"Initial number of features passed: {train_df.shape[1]}")
    logger.info("Starting computation")

    train_df_features = compute_features(train_df, target_col, train_to_dt)
    test_df_features = compute_features(test_df, target_col, test_to_dt)
    train_df_target = compute_target(
        compute_ready_data, target_col, target_train_month, keep_gap_month_churns
    )
    test_df_target = compute_target(
        compute_ready_data, target_col, target_test_month, keep_gap_month_churns
    )
    logger.info(f"Final number of features computed: {train_df_features.shape[1]}")
    logger.info(f"Length train data: {len(train_df_features)}")
    logger.info(f"Length test data: {len(test_df_features)}")
    logger.info("Computation done!")

    # As there are customer that leave between the month we use for training and the target month
    # We have to join the features and the targets and drop those that don't have target. By doing this,
    # we exclude customer that churned in gap month or those with no corresponding record in the target dataset.
    features_and_target_train = train_df_features.merge(
        train_df_target, on="customer_id", how="left"
    )
    features_and_target_test = test_df_features.merge(
        test_df_target, on="customer_id", how="left"
    )

    features_and_target_train = features_and_target_train[
        features_and_target_train[target_col[0]].notna()
    ]

    features_and_target_test = features_and_target_test[
        features_and_target_test[target_col[0]].notna()
    ]

    # Split train and test features + target (squeeze into 1D array)
    features = features_and_target_train.drop(columns=target_col + users_cols)
    features_test = features_and_target_test.drop(columns=target_col + users_cols)
    target = features_and_target_train[target_col].squeeze()
    target_test = features_and_target_test[target_col].squeeze()

    logger.info(f"Features: {features.columns.tolist()}")
    logger.info(f"Target: {target.name}")
    logger.info("Completed feature computation!")

    try:
        features.to_parquet(f"{save_features_path}/features.parquet", index=False)
        features_test.to_parquet(
            f"{save_features_path}/features_test.parquet", index=False
        )
        target.to_frame().to_parquet(f"{save_target_path}/target.parquet", index=False)
        target_test.to_frame().to_parquet(
            f"{save_target_path}/target_test.parquet", index=False
        )
        logger.info(f"Features saved on {save_features_path}")
        logger.info(f"Targets saved on {save_target_path}")
    except:
        logger.info(f"Features not saved on {save_features_path}")
        logger.info(f"Targets not saved on {save_target_path}")

    return features, target, features_test, target_test


def split_train_test(
    df: pd.DataFrame,
    train_from: pd.Series,
    train_to: pd.Series,
    test_from: pd.Series,
    test_to: pd.Series,
) -> (pd.DataFrame, pd.DataFrame):
    """
    Split data into train and test.
    Args:
        df: The clean dataset with the columns we want to use as features.
        train_from: The starting date of the training period.
        train_to: The ending date of the training period.
        test_from: The starting date of the testing period.
        test_to: The ending date of the testing period.

    Returns:
        DataFrame: Pandas DataFrame with training months only.
        DataFrame: Pandas DataFrame with testing months only.
    """
    # Create date col to mix month and year
    df["date"] = pd.to_datetime(
        df["YEAR"].astype(str) + "-" + df["MONTH"].astype(str) + "-01"
    )

    # Filter compute_data for the specific date intervals.
    df = df[(df["date"] >= train_from) & (df["date"] <= test_to)]

    train_df = df[(df["date"] >= train_from) & (df["date"] <= train_to)]
    test_df = df[(df["date"] >= test_from) & (df["date"] <= test_to)]

    return train_df, test_df


def compute_features(
    df: pd.DataFrame, target_col: list[str], train_to_dt: pd.Timestamp
) -> pd.DataFrame:
    """
    Compute the features and adds them to the df.
    Args:
        df: The clean dataset with the columns we want to use as features.
        target_col: Name of the target column.

    Returns:
        DataFrame: Pandas DataFrame with new computed variables.
    """

    # TO-DO: The rolling function is propagated backwards for each single month.
    # we just need it for the last one (but for that we need past data also). Didn't find
    # any option to do it with pandas

    df = df.drop(columns=target_col)

    df = df.sort_values(by=["customer_id", "date"])
    df = df.set_index("date")

    # Dynamically compute features for each col in transform_cols
    for col in transform_cols:
        df[f"{col}_prev_month"] = df.groupby("customer_id")[col].shift(1)
        # df[f"{col}_prev_month"] = df[f"{col}_prev_month"].fillna(0)
        df[f"{col}_avg_3_months"] = compute_x_months_avg(df, col, 3)
        df[f"{col}_avg_6_months"] = compute_x_months_avg(df, col, 6)
        df[f"{col}_std_3_months"] = compute_x_months_std(df, col, 3)
        df[f"{col}_std_6_months"] = compute_x_months_std(df, col, 6)

    # Filter only the computation backwards from the last month
    train_to_str = train_to_dt.strftime("%Y-%m-%d")
    df = df.loc[train_to_str]

    return df.reset_index(drop=True)


def compute_x_months_avg(
    df: pd.DataFrame, col_name: list[str], months: int
) -> pd.DataFrame:
    """
    Compute the mean of the last months for a column.
    Args:
        df: The clean dataset with the columns we want to use as features.
        col_name: Name of the column to compute
        months: Number of months we want to compute the feature back

    Returns:
        DataFrame: Pandas DataFrame the new computed column.
    """
    return (
        df.groupby("customer_id")[col_name]
        .rolling(window=months, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )


def compute_x_months_std(
    df: pd.DataFrame, col_name: list[str], months: int
) -> pd.DataFrame:
    """
    Compute the mean of the last months for a column.
    Args:
        df: The clean dataset with the columns we want to use as features.
        col_name: Name of the column to compute
        months: Number of months we want to compute the feature back

    Returns:
        DataFrame: Pandas DataFrame the new computed column.
    """
    return (
        df.groupby("customer_id")[col_name]
        .rolling(window=months, min_periods=1)
        .std()
        .reset_index(level=0, drop=True)
    )


def compute_target(
    df: pd.DataFrame,
    target_col: str,
    target_month: pd.Series,
    keep_gap_month_churns: bool = False,
) -> pd.DataFrame:
    """
    Compute the target column for a df.
    Args:
        df: The clean dataset with all the data.
        target_col: Name of the target column.
        target_month: The date where the target has to be computed.
        keep_gap_month_churns: A boolean parameter that determines the treatment of churns occurring in the gap month.
        If True, these churns are treated as actual churns (assigned a value of 1);
        otherwise, they are excluded from the analysis. Defaults to False.
    Returns:
        DataFrame: Pandas DataFrame with the customer_id and the target computed.
    """

    drop_churn_between_month = target_month - pd.DateOffset(months=1)

    target_df = df[
        (df["date"] == target_month) | (df["date"] == drop_churn_between_month)
    ][["customer_id"] + target_col + ["date"]]

    for col in target_col:
        target_df[col].fillna(0, inplace=True)
        target_df[col] = np.where(
            ((target_df[col] > 0) & (target_df["date"] == target_month)),
            1,
            target_df[col],
        )
        target_df[col] = np.where(
            ((target_df[col] > 0) & (target_df["date"] == drop_churn_between_month)),
            2,
            target_df[col],
        )

    if keep_gap_month_churns:
        # Convert all values that are 2 (gap month churns) into 1 (real churns)
        target_df[col] = np.where(target_df[col] == 2, 1, target_df[col])

    # Exclude the records that are still marked as 2 (gap month churns) if keep_gap_month_churns is False
    target_df = target_df[
        (target_df["NUM_DAYS_LINE_TYPE_FIXE_POST_DEA"] != 2)
        & (target_df["date"] != drop_churn_between_month)
    ][["customer_id"] + target_col]

    target_df[target_col] = target_df[target_col].astype("int")

    return target_df

In [6]:
query = f"""
    WITH all_periods AS (
    SELECT * 
    FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2022`
    UNION ALL 
    SELECT * 
    FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2023_1`
    ), 

    selectable_customer AS (
        SELECT customer_id
        FROM all_periods
        GROUP BY customer_id
    ), 

    customer_selected AS (
        SELECT customer_id AS selected_customer
        FROM selectable_customer
    WHERE MOD(ABS(FARM_FINGERPRINT(CAST(customer_id AS STRING))), 20) = 0
    )

    SELECT {", ".join(diff_cols + keep_cols + users_cols + target_col + transform_cols)}
    FROM all_periods
    INNER JOIN customer_selected
    ON customer_id = selected_customer
    WHERE IS_CUST_SEGM_RESI > 0
    AND IS_CUST_BILL_POST_CURR = TRUE
    AND CUST_BUNDLE_CURR = 'FMC'
    AND NUM_IMPAGOS = 0
    AND pago_final_0 IS NOT NULL
    """

get_initial_params()
raw_data = data_gathering(query, logger)
clean_data = data_cleaning(raw_data, logger)

INFO - Started querying data
INFO - Data succesfully retrieved! Length: 253077
INFO - Starting cleaning data
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Dropping column Import_Rest_quota_disp
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Should fillna
INFO - Dropping column NUM_FIX_PORT
INFO - Dropping column NUM_FIX_PORT_LAST_1_MONTH
INFO - Dropping column NUM_FIX_PORT_LAST_3_MONTHS
INFO - Dropping column NUM_FIX_PORT_LAST_6_MONTHS
INFO - Sh

In [ ]:
features, target, features_test, target_test = feature_computation(
    clean_data,
    train_from,
    train_to,
    logger,
    save_features_path=save_features_path,
    save_target_path=save_target_path,
)